# Mini-Batch GD and How Adverserial Examples affects its Performance

### Overview of this Notebook: <br>
#### 1. Import of Python libraries and the dataset<br>
#### 2. Tuning of the Hyperparameters of the Mini-Batch Optimiser<br>
#### 3. Training a Naive Model<br>
#### 4. Evaluate the Attack against the Naive Model<br>
#### 5. Training a Robust Model <br>
#### 6. Evaluate the Attack against the Robust Model <br>
#### 7. Comparision of the two performances <br>

## Begin by importing the relevant libraries

In [ ]:
from adversary import attack, protect
from net import Net
import numpy as np
from torch.optim import Optimizer
import torch
from training import training, testing, accuracy, tune_optimizer
from minibatch import MiniBatchOptimizer
import matplotlib.pyplot as plt
from data_utils import get_mnist, build_data_loaders
import json
from pathlib import Path
import random
import pandas as pd


## Get Data and Setup

In [ ]:
use_cuda = True
device = torch.device('cuda' if use_cuda and torch.cuda.is_available() else 'cpu')
train_dataset, test_dataset = get_mnist(normalize=True)

In [ ]:
epsilons = np.arange(0, 0.5, 0.05)
criterion = torch.nn.CrossEntropyLoss()
epochs = 10
batch_size = 16

## Hyperparameter Tuning

### To test the tuning, just set hyperparamter_tune to True. Otherwise, there is a JSON file with previous results

In [ ]:
hyperparamter_tune = True

This is the tuning setup. It initialises the Optimiser, a new Neural net and then generates a the True/False options for the flag for a Decreasing Learning rate.

In [ ]:
net_tune = Net().to(device)
mini_opt_tune = MiniBatchOptimizer(net_tune.parameters()) # Just using defaults
dec_lr_set =  [0]*1 + [1]*1
random.shuffle(dec_lr_set)
fp = 'mini_tuning.json'
if not hyperparamter_tune:
    results = []

If tuning is desired, then the following will use a custom function that takes only the training dataset, and uses it as the basic for new training/testing sets to prevent overfitting on the test data. The algorithm tries out every combination of elements in its search grid.

In [ ]:
if hyperparamter_tune:
    results = tune_optimizer(
    net_tune,
    train_dataset.data,
    train_dataset.targets,
    criterion,
    accuracy,
    device,
    MiniBatchOptimizer,
    epochs=10,
    search_grid={
        'lr': np.linspace(0.00001, 0.01, 1),
        'decreasing_lr': dec_lr_set,
    }, 
    batch_size=16
)

We then append any new results into the json file for the specific Optimiser so that we do not need to retread previous configurations.

In [ ]:
if Path(fp).exists():
    with open(fp, 'r') as f:
        old_results = json.load(f)

    results = old_results + results

with open(fp, 'w') as f:
    json.dump(results, f, indent=2)

# Select Best Hyperparamters
with open(fp, 'r') as f:
        old_results = json.load(f)

### Here we read out the best configuration, as determined by the Test Accuracy

In [ ]:
df_analysis = pd.DataFrame(results)
best_acc = 0.0
for index, row in df_analysis.iterrows():    
        trial_acc = row["metric_test"]
        if trial_acc > best_acc:
            best_acc = trial_acc
            learning_rate = round(row["lr"], 6)
            decreasing_lr = row["decreasing_lr"]

print("Best Accuracy was {}% with Learning Rate {} and Decreasing LR: {}".format(100*best_acc, learning_rate, decreasing_lr))


## Train the Naive Model

In [ ]:
net_naive = Net().to(device)
train_loader, test_loader = build_data_loaders(train_dataset, test_dataset, batch_size)

### Train and Test

In [ ]:
mini_opt_naive = MiniBatchOptimizer(net_naive.parameters(), lr=learning_rate, decreasing_lr=decreasing_lr)
loss_train, acc_train = training(net_naive, train_loader, mini_opt_naive, criterion, accuracy, epochs=epochs, device=device)
loss_test, acc_test = testing(net_naive, test_loader, criterion, accuracy, device=device)

## Attack Naive Model

In [ ]:
accuracy_naive= []
losses_naive= []

for eps in epsilons:
    loss_attack, acc_attack  = attack(net_naive, criterion, test_loader, epsilon=eps, device=device)
    accuracy_naive.append(acc_attack)
    losses_naive.append(loss_attack)

## Train the Robust Model

In [ ]:
robust_net = Net().to(device)
protect_epochs = epochs
protect_lr = learning_rate
protect_bz = batch_size
protect_dec_lr = decreasing_lr
prot_train_loader, prot_test_loader = build_data_loaders(train_dataset, test_dataset, protect_bz)
mini_opt_proc = MiniBatchOptimizer(robust_net.parameters(), lr=protect_lr, decreasing_lr=protect_dec_lr)

### Call the protect function to make the model robust

In [ ]:
robust_net = protect(robust_net, mini_opt_proc, criterion, prot_train_loader, prot_test_loader, device=device, epochs=protect_epochs)

## Attack the Robust Model

In [ ]:
accuracy_robust = []
losses_robust = []

for eps in epsilons:
    loss_attack, acc_attack = attack(robust_net, criterion, prot_train_loader, eps, device=device)
    accuracy_robust.append(acc_attack)
    losses_robust.append(loss_attack)

## Comparative Analysis of the Two Models

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(epsilons, accuracy_naive, "*-", c='blue', label='Naive Model')
plt.plot(epsilons, accuracy_robust, "*-", c='orange', label='Robust Model')

plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, 0.5, step=0.05))

plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.legend();